# info
- by: LeeHaEun
- start: 21.11.15 Mon
- end: 21.11.15 Mon
- review: 21.11.15 Mon

# theme:Trasfer learning

- 대규모 세트에서 훈련된 신경망 -> 이전에 본 적 없는 데이터 세트에 지식을 적용할 수 있다
- 최종 출력 레이어만 변경하면 된다 (아웃풋의 종류가 다르기 때문)
- 그 외는 변경 불가 (freezing model)
  - 사전 훈련된 모델의 변수 = 훈련 불가능
  - 마지막 분류 계층의 변수만 훈련되도록
- 훈련 시간 단축 가능!
- 가중치는 무작위로 초기화 -> 초기 오류 많다, 

## Mobilenet
- 메모리와 계산 리소스가 제한된 모바일 장치에서 사용하기 좋다
- imagenet을 이용해서 훈련되었다
- 텐서플로우 허브에서 사전 훈련된 모델을 다운 가능


# code_l06c01_tensorflow_hub_and_transfer_learning

1. Use a TensorFlow Hub model for prediction.
2. Use a TensorFlow Hub model for Dogs vs. Cats dataset.
3. Do simple transfer learning with TensorFlow Hub.

## import

In [ ]:
import tensorflow as tf

In [ ]:
import matplotlib.pylab as plt

import tensorflow_hub as hub #허브에서 모델 다운로드
import tensorflow_datasets as tfds

from tensorflow.keras import layers

In [ ]:
import logging
logger = tf.get_logger()
logger.setLevel(logging.ERROR)

## part1: 텐서플로우 허브의 모바일넷 사용(for prediction)

### classifier 다운로드

In [ ]:
CLASSIFIER_URL ="https://tfhub.dev/google/tf2-preview/mobilenet_v2/classification/2"
IMAGE_RES = 224

model = tf.keras.Sequential([
    hub.KerasLayer(CLASSIFIER_URL, input_shape=(IMAGE_RES, IMAGE_RES, 3))
])

### 1개의 이미지로만 돌려봅시다

In [ ]:
# 이미지 다운로드
import numpy as np
import PIL.Image as Image

grace_hopper = tf.keras.utils.get_file('image.jpg','https://storage.googleapis.com/download.tensorflow.org/example_images/grace_hopper.jpg')
grace_hopper = Image.open(grace_hopper).resize((IMAGE_RES, IMAGE_RES))
grace_hopper 

In [ ]:
# 픽셀 값이 0-1 사이에 있도록 정규화
grace_hopper = np.array(grace_hopper)/255.0
grace_hopper.shape

In [ ]:
# 트렌스퍼 한 모델에 러닝
result = model.predict(grace_hopper[np.newaxis, ...])
result.shape

In [ ]:
# 예측값
predicted_class = np.argmax(result[0], axis=-1)
predicted_class

In [ ]:
# 예측값의 라벨이 뭔지?
labels_path = tf.keras.utils.get_file('ImageNetLabels.txt','https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt')
imagenet_labels = np.array(open(labels_path).read().splitlines())

plt.imshow(grace_hopper)
plt.axis('off')
predicted_class_name = imagenet_labels[predicted_class]
_ = plt.title("Prediction: " + predicted_class_name.title())

## part2: 텐서플로우 허브 모델을 강아지-고양이 데이터에 적용

### dataset

In [ ]:
# 이미지 저장
(train_examples, validation_examples), info = tfds.load(
    'cats_vs_dogs', 
    with_info=True, 
    as_supervised=True, 
    split=['train[:80%]', 'train[80%:]'],
)

num_examples = info.splits['train'].num_examples
num_classes = info.features['label'].num_classes

In [ ]:
# 사이즈가 달라요 - 확인해봅시다
for i, example_image in enumerate(train_examples.take(3)):
  print("Image {} shape: {}".format(i+1, example_image[0].shape))

In [ ]:
# 리사이즈 해볼까요 - 모바일넷의 인풋은 (224,224)로 고정
def format_image(image, label):
  image = tf.image.resize(image, (IMAGE_RES, IMAGE_RES))/255.0
  return image, label

BATCH_SIZE = 32

train_batches      = train_examples.shuffle(num_examples//4).map(format_image).batch(BATCH_SIZE).prefetch(1)
validation_batches = validation_examples.map(format_image).batch(BATCH_SIZE).prefetch(1)

The .repeat() and steps_per_epoch here is not required, but saves ~15s per epoch, since the shuffle-buffer only has to cold-start once.

### run!

In [ ]:
image_batch, label_batch = next(iter(train_batches.take(1)))
image_batch = image_batch.numpy()
label_batch = label_batch.numpy()

result_batch = model.predict(image_batch)

predicted_class_names = imagenet_labels[np.argmax(result_batch, axis=-1)]
predicted_class_names

In [ ]:
# 확인해봅시다
plt.figure(figsize=(10,9))
for n in range(30):
  plt.subplot(6,5,n+1)
  plt.subplots_adjust(hspace = 0.3)
  plt.imshow(image_batch[n])
  plt.title(predicted_class_names[n])
  plt.axis('off')
_ = plt.suptitle("ImageNet predictions")

## part3: 간단한 트랜스퍼 러닝을 해봅시다

### feature_extacter

In [ ]:
URL = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/2"
feature_extractor = hub.KerasLayer(URL,
                                   input_shape=(IMAGE_RES, IMAGE_RES,3))

In [ ]:
feature_batch = feature_extractor(image_batch)
print(feature_batch.shape)
#-> 32, 1280
# 32:이미지 개수
# 1280: 뉴런의 개수

In [ ]:
# 추출 레이어의 변수 동결 -> classifier layer 만 변경
feature_extractor.trainable = False

### classifier layer 수정

In [ ]:
model = tf.keras.Sequential([
  feature_extractor,
  layers.Dense(2) #output 2개로 변경
])

model.summary()

### 모델 학습시키기

In [ ]:
model.compile(
  optimizer='adam',
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])

EPOCHS = 6
history = model.fit(train_batches,
                    epochs=EPOCHS,
                    validation_data=validation_batches)

#-> 검증 정확도 97%
# 모바일넷은 전문가에 의해 신중하게 설계되고, 대규모 데이터세트(이미지넷)에 의해 교육되었기 때문

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(EPOCHS)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

validation performance > training performance
1. v는 epoch말기에 측정, t는 전체의 평균 값
2. 이미 개와 고양이 이미지에 대해 교육받은 모바일넷의 많은 부분을 재사용하고 있기 때문
  - t에 대해서는 image augumentation 하지만, v는 안한다.
  - = t가 더 어려울 수 있다

### Check the predictions

In [ ]:
class_names = np.array(info.features['label'].names)
class_names

In [ ]:
predicted_batch = model.predict(image_batch)
predicted_batch = tf.squeeze(predicted_batch).numpy()
predicted_ids = np.argmax(predicted_batch, axis=-1)
predicted_class_names = class_names[predicted_ids]
predicted_class_names

In [ ]:
print("Labels: ", label_batch)
print("Predicted labels: ", predicted_ids)

In [ ]:
plt.figure(figsize=(10,9))
for n in range(30):
  plt.subplot(6,5,n+1)
  plt.subplots_adjust(hspace = 0.3)
  plt.imshow(image_batch[n])
  color = "blue" if predicted_ids[n] == label_batch[n] else "red"
  plt.title(predicted_class_names[n].title(), color=color)
  plt.axis('off')
_ = plt.suptitle("Model predictions (blue: correct, red: incorrect)")